In [1]:
### Choosing genes for analysis ###

### Citations of tools ###
# gget: Luebbert, L., & Pachter, L. (2023). Efficient querying of genomic reference databases with gget. Bioinformatics. https://doi.org/10.1093/bioinformatics/btac836
# pybiomart: https://github.com/jrderuiter/pybiomart

In [2]:
import pandas as pd
import gget
import json

In [48]:
# data file downloaded from https://www.encodeproject.org/experiments/ENCSR792OIJ/
# Accession ENCFF928NYA under processed data (tsv file) 
# Data added 2020-11-04
df = pd.read_table("ENCFF928NYA.tsv")
columns = ['gene_id', 'TPM', 'FPKM', 'transcript_id(s)']
df = pd.DataFrame(df, columns=columns)

In [49]:
df.head()

,gene_id,TPM,FPKM,transcript_id(s)
0,10904,0.0,0.0,10904
1,12954,0.0,0.0,12954
2,12956,0.0,0.0,12956
3,12958,0.0,0.0,12958
4,12960,0.0,0.0,12960


In [5]:
expressed_df = df.query("TPM!=0.0")

high_thresh = 200
high_thresh_df = expressed_df.query("TPM>@high_thresh")

low_thresh = 10
low_thresh_df = expressed_df.query("TPM>5 & TPM<@low_thresh")
lowest_thresh_df = expressed_df.query("TPM<5")

# mid_thresh_df = high_thresh_df.query("TPM<20000")
# mid_thresh_df = mid_thresh_df.drop(index=[59429, 59431, 59456, 59471, 59484, 59493, 59500, 59524])

In [6]:
# print(high_thresh_df.to_string())
#print(len(mid_thresh_df.index))

In [7]:
expressed_df.head()

,gene_id,TPM,FPKM
46,13023,41.07,49.61
47,13024,16.31,19.70
465,30958,43.49,52.53
471,30964,52.81,63.79
649,ENSG00000000003.14,0.05,0.06


In [8]:
# rows were selected by looking at the entire dataframe and were chosen for varying expression levels
high_rows = [941, 1965, 4142, 10644, 17808, 24929, 48356, 53485, 54599, 55026, 55115, 57853]
low_rows = [657, 926]
lowest_rows = [649, 652]

chosen_rows = high_rows + low_rows + lowest_rows

In [9]:
chosen_rows_df = pd.DataFrame(df.loc[chosen_rows])

chosen_rows_gene_ids = []
for index, row in chosen_rows_df.iterrows():
    chosen_rows_gene_ids.append(row['gene_id'])
    
chosen_rows_df.head(n=len(chosen_rows_df.index))

,gene_id,TPM,FPKM
941,ENSG00000011304.19,342.64,413.86
1965,ENSG00000075624.14,1567.70,1893.59
4142,ENSG00000108107.14,1401.16,1692.44
10644,ENSG00000156508.17,5000.06,6039.46
17808,ENSG00000196565.14,4169.72,5036.51
24929,ENSG00000222328.1,7046.27,8511.03
48356,ENSG00000263934.4,13928.59,16824.04
53485,ENSG00000274012.1,94300.79,113903.82
54599,ENSG00000276168.1,149831.97,180978.68
55026,ENSG00000277027.1,23024.05,27810.24


In [10]:
### METHOD USING GGET ###

In [11]:
res = gget.info(chosen_rows_gene_ids, json=True, uniprot=False)

Fri Jun 30 03:35:01 2023 INFO We noticed that you passed a version number with your Ensembl ID.
Please note that gget info will always return information linked to the latest Ensembl ID version (see 'ensembl_id').


In [12]:
ordered_gene_names = [None] * len(chosen_rows_gene_ids)

for key in res.keys():
    for i in range(len(chosen_rows_gene_ids)):
        if chosen_rows_gene_ids[i].startswith(key):
            ordered_gene_names[i] = res[key]['ensembl_gene_name']

In [13]:
chosen_rows_df.insert(1, "gene name (ensembl)", ordered_gene_names, True)
chosen_rows_df.head(n=len(chosen_rows_df.index))

,gene_id,gene name (ensembl),TPM,FPKM
941,ENSG00000011304.19,PTBP1,342.64,413.86
1965,ENSG00000075624.14,ACTB,1567.70,1893.59
4142,ENSG00000108107.14,RPL28,1401.16,1692.44
10644,ENSG00000156508.17,EEF1A1,5000.06,6039.46
17808,ENSG00000196565.14,HBG2,4169.72,5036.51
24929,ENSG00000222328.1,RNU2-2P,7046.27,8511.03
48356,ENSG00000263934.4,SNORD3A,13928.59,16824.04
53485,ENSG00000274012.1,RN7SL2,94300.79,113903.82
54599,ENSG00000276168.1,RN7SL1,149831.97,180978.68
55026,ENSG00000277027.1,RMRP,23024.05,27810.24


In [14]:
# However, we need the RefSeq ID beginning in NM_ to use PaintSHOP, and gget doesn't let us do that. 
# So, the following method uses pybiomart which uses the ensembl API and MANE (Matched Annotation from the NCBI and EMBL-EBI) which correlates RefSeq IDs to Ensembl IDs.

In [15]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl',
                  host='http://www.ensembl.org')

In [16]:
refseq_results = []

# Gets the gene name and the RefSeq ID associated with the canonical (most common) transcript
for i in chosen_rows_gene_ids:
    gene_id = i.split(".")[0] # remove the version number
    result = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name', 'external_transcript_name', 'transcript_mane_select', 'transcript_is_canonical', 'ensembl_transcript_id'],
                  filters={'link_ensembl_gene_id': [gene_id], 'transcript_is_canonical' : True})
    refseq_results.append(result)
    
to_extract = chosen_rows_df.reset_index()

In [17]:
refseq_df = pd.concat(refseq_results, ignore_index = True)
refseq_df.insert(2, "TPM", to_extract["TPM"])
refseq_df.insert(3, "FPKM", to_extract["FPKM"])
refseq_df['Ensembl Canonical'] = refseq_df['Ensembl Canonical'].astype(bool)  
print(refseq_df.to_string())
refseq_df.head(n=len(refseq_df.index))

     Gene stable ID Gene name        TPM       FPKM Transcript name RefSeq match transcript (MANE Select)  Ensembl Canonical Transcript stable ID
0   ENSG00000011304     PTBP1     342.64     413.86       PTBP1-203                           NM_002819.5               True      ENST00000356948
1   ENSG00000075624      ACTB    1567.70    1893.59        ACTB-217                           NM_001101.5               True      ENST00000646664
2   ENSG00000108107     RPL28    1401.16    1692.44       RPL28-201                           NM_000991.5               True      ENST00000344063
3   ENSG00000156508    EEF1A1    5000.06    6039.46      EEF1A1-201                           NM_001402.6               True      ENST00000309268
4   ENSG00000196565      HBG2    4169.72    5036.51        HBG2-201                           NM_000184.3               True      ENST00000336906
5   ENSG00000222328   RNU2-2P    7046.27    8511.03     RNU2-2P-201                                   NaN               True

,Gene stable ID,Gene name,TPM,FPKM,Transcript name,RefSeq match transcript (MANE Select),Ensembl Canonical,Transcript stable ID
0,ENSG00000011304,PTBP1,342.64,413.86,PTBP1-203,NM_002819.5,True,ENST00000356948
1,ENSG00000075624,ACTB,1567.70,1893.59,ACTB-217,NM_001101.5,True,ENST00000646664
2,ENSG00000108107,RPL28,1401.16,1692.44,RPL28-201,NM_000991.5,True,ENST00000344063
3,ENSG00000156508,EEF1A1,5000.06,6039.46,EEF1A1-201,NM_001402.6,True,ENST00000309268
4,ENSG00000196565,HBG2,4169.72,5036.51,HBG2-201,NM_000184.3,True,ENST00000336906
5,ENSG00000222328,RNU2-2P,7046.27,8511.03,RNU2-2P-201,NaN,True,ENST00000410396
6,ENSG00000263934,SNORD3A,13928.59,16824.04,SNORD3A-201,NaN,True,ENST00000584923
7,ENSG00000274012,RN7SL2,94300.79,113903.82,RN7SL2-201,NaN,True,ENST00000490232
8,ENSG00000276168,RN7SL1,149831.97,180978.68,RN7SL1-201,NaN,True,ENST00000618786
9,ENSG00000277027,RMRP,23024.05,27810.24,RMRP-201,NaN,True,ENST00000363046


In [18]:
# mergedRes = pd.merge(expressed_df, refseq_df, on ='gene_id')
# mergedRes.head(len(mergedRes.index))

In [19]:
# Note that several of the genes have no matching RefSeq ID. This is because they are not mRNA, but other kinds of RNA (i.e. snRNA or rRNA). 
# Ensembl doesn't already have the RefSeq IDs available for this. --> see PartialAnalysis notebook to filter these out

In [20]:
valid_seqs = []
# Export to file to use in PaintSHOP
f = open("refseqs_sample.txt", "w")
for index, row in refseq_df.iterrows():
    if str(row['RefSeq match transcript (MANE Select)']) != 'nan':
        valid_seqs.append(str(row['RefSeq match transcript (MANE Select)']))
        f.write(str(row['RefSeq match transcript (MANE Select)']) + '\n')
f.close()

In [21]:
# Then we use PaintSHOP to make probes: https://doi.org/10.1038/s41592-021-01187-3

In [41]:
data = pd.read_table('230629-PaintSHOP-probes.txt')

In [42]:
#data.head(len(data.index))

In [43]:
mask = (data['sequence'].str.len() <= 31)
data = data.loc[mask]
data = data.sort_values(['gene_id', 'off_target'], ascending=[True, True])
data.head(n=300)

#data.query("'gene_id'=='EEF1A1'").head(n=20)

,refseq,chrom,start,stop,sequence,Tm,on_target,off_target,repeat_seq,prob,max_kmer,probe_strand,transcript_id,gene_id
14,NM_001101,chr7,5528340,5528369,ATGACCTGGCCGTCAGGCAGCTCGTAGCTC,46.30,100.000,0.000,0,0.064,2,+,NM_001101,ACTB
15,NM_001101,chr7,5528658,5528688,AGGGATAGCACAGCCTGGATAGCAACGTACA,42.32,100.000,0.000,0,0.277,2,+,NM_001101,ACTB
18,NM_001101,chr7,5529286,5529315,CCCAGTTGGTGACGATGCCGTGCTCGATGG,45.84,99.194,99.194,0,0.226,3,+,NM_001101,ACTB
17,NM_001101,chr7,5528567,5528596,GGGGAGGGCATACCCCTCGTAGATGGGCAC,46.15,97.338,100.000,0,0.051,2,+,NM_001101,ACTB
13,NM_001101,chr7,5528310,5528339,GGGCAGCGGAACCGCTCATTGCCAATGGTG,46.93,97.920,166.778,0,0.083,2,+,NM_001101,ACTB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,NM_003270,chrX,100635578,100635608,AAGCTCGGCAGGTAGCAAAACAACCAAAGGT,42.60,100.000,0.000,0,0.239,0,+,NM_003270,TSPAN6
249,NM_003270,chrX,100636686,100636715,GGACGCCATGACTAGCCCGAGACCCTGCAC,46.66,98.682,0.000,0,0.351,0,+,NM_003270,TSPAN6
250,NM_003270,chrX,100635715,100635745,ATGCCAACTGCAAGAAGGATAACGCCAGTGA,42.11,100.000,0.000,0,0.168,2,+,NM_003270,TSPAN6
255,NM_003270,chrX,100627400,100627429,CCCCAGCCTGTGCCACTAAGGGAAACTCAC,43.90,100.000,0.000,0,0.181,2,+,NM_003270,TSPAN6


In [44]:
gene_probes_dict = {}

for i in valid_seqs:
    gene = data.query("refseq=='" + i.split(".")[0] + "'")
    gene_probes_dict[i] = gene
    print(i, len(gene.index))

NM_002819.5 40
NM_001101.5 6
NM_000991.5 0
NM_001402.6 0
NM_000184.3 6
NM_001498.4 10
NM_001286.5 61
NM_003270.4 6
NM_020423.7 18


In [45]:
# HGB1: 
# 2: one for HGB1 and another for HGB2 (one off target on ch2 --> not a gene)
# 6: one for HGB1 and another for HGB2
# 12: one for HGB1 and another for HGB2
# 0: one for HGB1 and another for HGB2 (off target gene on ch6)
# 12, 6, 2, 7, 8 (7, and 8 made smaller to fit)

# ACTB:
# 14: ch7 -> match to ACTB, ch6 no gene, ch2 POTEI off target
# 18: of 7 results, 2 no gene, 1 is ACTB, 4 off target genes
# 17: has ACTB, 3 no gene
# 13: has ACTB, ACTG1 and OMIM genes on ch17, 2 no gene
# 16: has ACTB, lots of overlapping genes

# TSPAN6:
# 257 has off target gene with correct TSPAN6
# 249, 242, and 255 are good

# None for EEF1A1

# PTBP1:
# 183, 186, 187, 188, 190
# 185 is bad
# 233, 210, 234, 219, 201, 216, 214, 236, 224 are bad

# GCLC:
# 156, 167, 169, 175, 177
# 145, 176, 165 are bad

# SCYL3:
# 289 and 294 are bad
# 268, 275, 276, 280, 284

# CLCN6:
# 106, 121, 109, 40 are bad
# 46, 54, 102 have no gene


In [46]:
to_print = gene_probes_dict["NM_000184.3"]
for i, row in to_print.iterrows():
    print("> ID" + str(i))
    print(row["sequence"])

> ID7
ATCATGGGCAGTGAGCTCAGTGGTATCTGGA
> ID0
CTTTGGGGTTGCCCATGATGGCAGAGGCAG
> ID12
GGACAGGGCACTGGCCACTCCAGTCACCAT
> ID6
AGTTCACTCAGCTGGGCAAAGGTGCCCTTG
> ID2
CTTCTGCCAGGAAGCCTGCACCTCAGGGGT
> ID8
AGGACAGGTTGCCAAAGCTGTCAAAGAACCT


In [47]:
to_print.head(n=70)

,refseq,chrom,start,stop,sequence,Tm,on_target,off_target,repeat_seq,prob,max_kmer,probe_strand,transcript_id,gene_id
7,NM_000184,chr11,5253261,5253291,ATCATGGGCAGTGAGCTCAGTGGTATCTGGA,42.22,96.905,96.905,0,0.298,3,+,NM_000184,HBG2
0,NM_000184,chr11,5254426,5254455,CTTTGGGGTTGCCCATGATGGCAGAGGCAG,44.80,97.438,97.438,0,0.115,3,+,NM_000184,HBG2
12,NM_000184,chr11,5253292,5253321,GGACAGGGCACTGGCCACTCCAGTCACCAT,46.55,100.000,98.154,0,0.081,2,+,NM_000184,HBG2
6,NM_000184,chr11,5254332,5254361,AGTTCACTCAGCTGGGCAAAGGTGCCCTTG,44.24,99.200,99.200,0,0.079,2,+,NM_000184,HBG2
2,NM_000184,chr11,5253322,5253351,CTTCTGCCAGGAAGCCTGCACCTCAGGGGT,46.58,100.000,100.000,0,0.138,3,+,NM_000184,HBG2
8,NM_000184,chr11,5254456,5254486,AGGACAGGTTGCCAAAGCTGTCAAAGAACCT,42.08,100.000,100.000,0,0.137,2,+,NM_000184,HBG2
